# Paso 4: Data mining

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [23]:
dfcat = pd.read_csv(r'F:\2022-1\Analisis de datos\Tarea Academica\dfcat_post_trans.csv')
dfnum = pd.read_csv(r'F:\2022-1\Analisis de datos\Tarea Academica\dfnum_sin_outliers.csv')

In [24]:
dfunion = pd.concat([dfnum,dfcat], axis = 1)

In [25]:
# Eliminamos columnas sin informacion relevante o que no podremos usar dentro de los modelos de clasificacion
dfunion.drop(columns = ["popularity","key", "id", "id_artists", "release_year"], inplace = True)
dfunion.head()

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,explicit,mode,time_signature,Name_Length,words_name,release_month,release_day,release_trim,list_pop,list_key
0,5.103472,0.645,0.4450,-13.338,-0.344862,0.674,-0.127844,0.151,0.127,104.851,0,1,3,5.0,1.0,2,22,1,0,0
1,4.992111,0.695,0.2630,-22.136,-1.258605,0.797,-3.000000,0.148,0.655,102.009,0,1,1,31.0,5.0,6,1,2,0,0
2,5.259211,0.434,0.1770,-21.180,-1.282329,0.994,-1.642065,0.212,0.457,130.418,0,1,5,30.0,5.0,3,21,1,0,2
3,5.247745,0.321,0.0946,-27.961,-1.289037,0.995,-2.165238,0.104,0.397,169.980,0,1,3,26.0,4.0,3,21,1,0,0
4,5.212401,0.402,0.1580,-16.900,-1.397940,0.989,-0.882729,0.311,0.196,103.220,0,0,4,16.0,4.0,1,1,1,0,1


## Separar en variable dependiente y variable independiente

In [26]:
y = dfunion["list_pop"]
x = dfunion[['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'explicit', 'mode', 'time_signature', 'Name_Length', 'words_name',
       'release_month', 'release_day', 'release_trim','list_key']]

## Sample splitting

In [27]:
# dividimos en conjunto de train, test y validation
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=100, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=200, stratify=y_train)

## Balanceo de Datos

In [28]:
pd.DataFrame(y_train).value_counts(normalize=True)*100

list_pop
0           73.131456
1           26.868544
dtype: float64

In [29]:
pd.DataFrame(y_train).value_counts()

list_pop
0           241336
1            88667
dtype: int64

Nuestra variable no está tan desbalanceada

In [30]:
nas = pd.DataFrame(x_train.isnull().sum()).sort_values(0,ascending=False)
nas.columns = ['nas']
nas['nas%'] = round(nas['nas']/x_train.shape[0], 2)
nas 

,nas,nas%
words_name,48,0.0
Name_Length,48,0.0
duration_ms,0,0.0
explicit,0,0.0
release_trim,0,0.0
release_day,0,0.0
release_month,0,0.0
time_signature,0,0.0
mode,0,0.0
tempo,0,0.0


No tenemos valores nulos, asi que podemos generar datos balanceados

In [31]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from collections import Counter 

os_us = SMOTETomek(random_state=2022, sampling_strategy='all' ) #

x_train_smote, y_train_smote = os_us.fit_resample(x_train, y_train)

#x_train_all_smote, y_train_all_smote = os_us.fit_resample(x_train, y_train)

ValueError: Input X contains NaN.
SMOTETomek does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')

In [38]:
print(y_train.value_counts(normalize=True)*100) #antes del balanceo
print(y_train.value_counts(normalize=True)*100) 

print(y_train_smote.value_counts(normalize=True)*100) #luego de aplicar la tecnica smote
print(y_train_smote.value_counts(normalize=True)*100)

0    73.128421
1    26.871579
Name: list_pop, dtype: float64
0    73.128421
1    26.871579
Name: list_pop, dtype: float64
1    50.0
0    50.0
Name: list_pop, dtype: float64
1    50.0
0    50.0
Name: list_pop, dtype: float64


## Árbol de clasificación CART

### Generamos arboles de decision con datos balanceados y sin balancear

Elegimos el modelo

In [39]:
from sklearn.tree import DecisionTreeClassifier

# Creacion del modelo 
modelo1 = DecisionTreeClassifier(criterion="gini", random_state=2022)
modelo1b = DecisionTreeClassifier(criterion="gini", random_state=2022)
# Indicador gini ayuda a medir la calidad de la división de los datos

Entrenamos el modelo 

In [40]:
# Entrenamiento del modelo 
modelo1.fit(x_train, y_train)

DecisionTreeClassifier(random_state=2022)

In [41]:
# Entrenamiento del modelo 
modelo1b.fit(x_train_smote, y_train_smote)

DecisionTreeClassifier(random_state=2022)

Realizamos predicciones con datos de test

In [42]:
# Realizamos predicciones con datos de test
y_predict_modelo1 = modelo1.predict(x_test)
y_predict_modelo1

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [43]:
# Realizamos predicciones con datos de test
y_predict_modelo1b = modelo1b.predict(x_test)
y_predict_modelo1b

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [44]:
# Se calcula el accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test, y_predict_modelo1) 

0.7337351944412244

In [45]:
# Se calcula el accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test, y_predict_modelo1b) 

0.7195450423113378

In [46]:
from sklearn.model_selection import cross_val_score

scores_res = cross_val_score(modelo1, x_train_smote, y_train_smote, cv=10)
print("Accuracy de los 10 folds:", scores_res)
print("Accuracy Mean: {:.3f} (std: {:.3f})".format(scores_res.mean(), scores_res.std()) )

Accuracy de los 10 folds: [0.69955997 0.69591764 0.69841671 0.75709533 0.83971282 0.83836534
 0.8353335  0.83562826 0.83564932 0.83613357]
Accuracy Mean: 0.787 (std: 0.063)


In [47]:
from sklearn.model_selection import cross_val_score

scores_res = cross_val_score(modelo1b, x_train, y_train, cv=10)
print("Accuracy de los 10 folds:", scores_res)
print("Accuracy Mean: {:.3f} (std: {:.3f})".format(scores_res.mean(), scores_res.std()) )

Accuracy de los 10 folds: [0.73209686 0.73146044 0.73378591 0.73454358 0.73378591 0.73572554
 0.73230088 0.73154322 0.73036126 0.72936113]
Accuracy Mean: 0.732 (std: 0.002)


Matriz de confusion

In [48]:
# Confusion matrix modelo1
print(pd.crosstab(y_test, y_predict_modelo1))

col_0         0      1
list_pop              
0         87166  20077
1         18971  20437


In [49]:
# Confusion matrix modelo1b
print(pd.crosstab(y_test, y_predict_modelo1b))

col_0         0      1
list_pop              
0         83542  23701
1         17428  21980


In [50]:
# Se genera el arbol de clasificacion - modelo1
target = list(y_test.unique())
feature_names = list(x_train.columns)

In [51]:
# Se genera el arbol de clasificacion - modelo1b
target = list(y_test.unique())
feature_names = list(x_train_smote.columns)

Falta graficar arbol

In [52]:
from sklearn import tree
from sklearn.tree import graphviz
graphviz.Source(tree.export_graphviz(modelo, 
                                     feature_names = feature_names, 
                                     class_names = target, 
                                     filled=True, rounded=True))  

ImportError: cannot import name 'graphviz' from 'sklearn.tree' (c:\Users\c3318\anaconda3\envs\Jypiter_with_R\lib\site-packages\sklearn\tree\__init__.py)

Guardamos el modelo

In [ ]:
import pickle 
pickle.dump(modelo1, open('modelo1', 'wb')) 

In [ ]:
import pickle 
pickle.dump(modelo1b, open('modelo1b', 'wb')) 

Realizamos predicciones en el modelo 1 

In [ ]:
import_modelo1 = pickle.load(open('modelo1', 'rb'))
result = import_modelo1.predict(x_test)
pd.DataFrame(result).value_counts() 

In [ ]:
import_modelo1b = pickle.load(open('modelo1b', 'rb'))
result = import_modelo1b.predict(x_test)
pd.DataFrame(result).value_counts() 

## Regresión logísitica

In [54]:
#Escalamiento de las variables
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Se establecen las variables que el GridSearch va buscar
parameters = {
        "max_iter": [50, 100],
        "random_state": [0]
    }

#Definimos el modelo
model2 = LogisticRegression()

#Se prepara el Stratified Cross-Validation
grid_search = GridSearchCV(model2, param_grid=parameters, cv=5, n_jobs=-1, scoring="accuracy")

In [56]:
#Se entrena el modelo
grid_search.fit(x_train, y_train)

c:\Users\c3318\anaconda3\envs\Jypiter_with_R\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'max_iter': [50, 100], 'random_state': [0]},
             scoring='accuracy')

In [57]:
model2 = grid_search.best_estimator_

model2

LogisticRegression(random_state=0)

In [58]:
best_params = grid_search.best_params_

best_params

{'max_iter': 100, 'random_state': 0}

Evaluamos

In [60]:
#Revisamos el accuracy en el conjunto de prueba
model2.score(x_test, y_test)

0.7812902741883793

In [61]:
# Realizamos predicciones con datos de test
y_predict_modelo2 = model2.predict(x_test)
y_predict_modelo2

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [74]:
# Confusion matrix modelo1
print(pd.crosstab(y_test, y_predict_modelo2))

col_0         0      1
list_pop              
0         97156  10087
1         21987  17421


## Análisis de los modelos

In [ ]:
def indicadores(TP, FP, TN, FN):
    Sensibility = TP/(TP+FN)
    print( 'Sensibilidad es:',Sensibility)
    Specificity = TN/(TN+FP)
    print('Especificidad es:', Specificity)
    Precision = TP/(TP+FP)
    print('Precision es:', Precision)
    Recall = TP/(TP+FN)
    print('Recall es:', Recall)
    F1 = 2*(Precision*Recall)/(Precision+Recall)
    print('F1 es:', F1)
    return Sensibility, Specificity, Precision, Recall, F1

### Modelo de regresión logística

In [83]:
modelo_logit = indicadores(17421, 10087, 97156, 21987)

Sensibilidad es: 0.4420676004872107
Especificidad es: 0.9059425790028254
Precision es: 0.6333066744219863
Recall es: 0.4420676004872107
F1 es: 0.5206826469005917


In [85]:
modelo_logit
# Sensibility, Specificity, Precision, Recall, F1 

(0.4420676004872107,
 0.9059425790028254,
 0.6333066744219863,
 0.4420676004872107,
 0.5206826469005917)

In [91]:
#Revisamos el accuracy en el conjunto de prueba
model2.score(x_test, y_test)

0.7812902741883793

### Modelo de árbol de clasificación CART sin balanceo de datos

In [92]:
# Confusion matrix modelo1
print(pd.crosstab(y_test, y_predict_modelo1))

col_0         0      1
list_pop              
0         87166  20077
1         18971  20437


In [93]:
# indicadores(TP, FP, TN, FN)
modelo_arbol_cart_nobal = indicadores(20437, 20077, 87166, 18971)

Sensibilidad es: 0.5186002842062526
Especificidad es: 0.8127896459442574
Precision es: 0.5044429086241793
Recall es: 0.5186002842062526
F1 es: 0.5114236380470959


In [94]:
# Se calcula el accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test, y_predict_modelo1) 

0.7337351944412244

### Modelo de árbol de clasificación CART con balanceo de datos

In [87]:
# Confusion matrix modelo1
print(pd.crosstab(y_test, y_predict_modelo1b))

col_0         0      1
list_pop              
0         83542  23701
1         17428  21980


In [90]:
# indicadores(TP, FP, TN, FN)
modelo_arbol_card_balanc = indicadores(21980, 23701, 83542, 17428)

Sensibilidad es: 0.5577547706049533
Especificidad es: 0.7789972305884766
Precision es: 0.4811628466977518
Recall es: 0.5577547706049533
F1 es: 0.5166355228055329


In [95]:
# Se calcula el accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test, y_predict_modelo1b) 

0.7195450423113378